In [1]:
import pandas as pd
import sys
from sklearn.pipeline import Pipeline
import numpy as np
import string
import os
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

from xgboost import XGBRegressor

In [2]:
os.path.abspath('..')
sys.path.append(os.path.abspath('..'))


In [4]:
import importlib


import board_helpers
from board_helpers.move_info import MoveInfo
from board_helpers.tiles_counter import TilesCounter
from board_helpers.board_consts import TileType
importlib.reload(board_helpers)
importlib.reload(board_helpers.move_info)
importlib.reload(board_helpers.tiles_counter)
importlib.reload(board_helpers.board_consts)



import transformers
from transformers.columns_setter_transformer import ColumnsSetterTransformer
from transformers.extract_set_column_transformer import ExtractSetColumnsTransformer
from transformers.name_dropper_transformer import NameDropperTransformer
from transformers.select_transformer import SelectTransformer
from transformers.select_by_index_transformer import SelectByIndexTransformer
from transformers.series_from_group_transformer import SeriesFromGroupTransformer
from transformers.map_set_transformer import MapSetTransformer
from transformers.one_hot_encoder_transformer import OneHotEncoderTransformer
from transformers.add_to_dict_transformer import AddToDictTransformer
from transformers.get_from_dict_transformer import GetFromDictTransformer
from transformers.select_by_mask_transformer import SelectByMaskTransformer
from transformers.select_features_wrapper import SelectFeaturesWrapper
from transformers.my_turns_transformation import MyTurnsTransformation
importlib.reload(transformers)
importlib.reload(transformers.columns_setter_transformer)
importlib.reload(transformers.extract_set_column_transformer)
importlib.reload(transformers.name_dropper_transformer)
importlib.reload(transformers.select_transformer)
importlib.reload(transformers.select_by_index_transformer)
importlib.reload(transformers.series_from_group_transformer)
importlib.reload(transformers.map_set_transformer)
importlib.reload(transformers.one_hot_encoder_transformer)
importlib.reload(transformers.add_to_dict_transformer)
importlib.reload(transformers.get_from_dict_transformer)
importlib.reload(transformers.select_by_mask_transformer)
importlib.reload(transformers.select_features_wrapper)
importlib.reload(transformers.my_turns_transformation)


import functions
from functions.bot_extractor import BotExtarctor
from functions.is_bot_extractor import IsBotExtarctor
importlib.reload(functions)
importlib.reload(functions.is_bot_extractor)
importlib.reload(functions.bot_extractor)

import processors
from processors.basic_pre_processor import BasicPreProcessor
importlib.reload(processors)
importlib.reload(processors.basic_pre_processor)

import builders
from builders.preprocessor_builder import PreprocessorBuilder
from builders.reg_pipe_builder import RegPipeBuilder
importlib.reload(builders)
importlib.reload(builders.preprocessor_builder)
importlib.reload(builders.reg_pipe_builder)

import searcher
import searcher_results_orgenizer
from searcher import Searcher
from searcher_results_orgenizer import SearcherResultsOrgenizer
importlib.reload(searcher)
importlib.reload(searcher_results_orgenizer)







<module 'searcher_results_orgenizer' from 'c:\\Users\\Bina4\\Desktop\\Guy_hafifa\\scrabble\\searcher_results_orgenizer.py'>

In [5]:
games = pd.read_csv('../data/games.csv', index_col='game_id')
train = pd.read_csv('../data/train.csv', index_col='game_id')
turns = pd.read_csv('../data/fs_turns.csv', index_col='game_id')

In [6]:
G_NAME = 'games'
T_NAME = 'turns'
DATA_NAME = 'train'

names = ['BetterBot', 'STEEBot', 'HastyBot']

prePipe = Pipeline([
                     ('get_relavent_games',  SelectByIndexTransformer(train.index.unique(), target=G_NAME)),
                     ('train_set_is_player', ExtractSetColumnsTransformer({'is_player': IsBotExtarctor(names, 'nickname', True)},
                                                                           src=DATA_NAME, dest=DATA_NAME)),
                     ('get_bot_rating', ExtractSetColumnsTransformer({'bot_rating': lambda train: train[~train['is_player']]['rating']},
                                                                      src=DATA_NAME, dest=G_NAME)),
                     ('get_bots_names', ExtractSetColumnsTransformer({'bot_name': lambda train: train[~train['is_player']]['nickname']},
                                                                     src=DATA_NAME, dest=G_NAME)),
                    ('data_drop_bot_rating', SelectByMaskTransformer('is_player', target=DATA_NAME)),
                    ])

In [7]:
preprocessor = PreprocessorBuilder(games, G_NAME, turns, T_NAME, prePipe).build()
n_games, n_turns, n_ratings = preprocessor.process(train, DATA_NAME)

In [8]:
n_data = n_games.merge(n_turns, left_index=True, right_index=True)

In [16]:
features = np.load('../feature_selection_consts/columns.npy', allow_pickle=True)

featureTransformers = [('extract_first', ColumnsSetterTransformer({'is_bot_first': IsBotExtarctor(names, name_col='first')})),
                       ('hot', OneHotEncoderTransformer({'time_control_name': games['time_control_name'].unique(),
                                                          'game_end_reason':  games['game_end_reason'].unique(),
                                                          'lexicon':  games['lexicon'].unique(),
                                                          'rating_mode':  games['rating_mode'].unique(),
                                                          'bot_name': names}
                                                        )),
                       ('select_col', SelectTransformer(features)),                                              
                      ]

In [18]:
reg_pipe = Pipeline(featureTransformers)
t_data = reg_pipe.transform(n_data)

In [17]:
t_data

,initial_time_seconds,max_overtime_minutes,game_duration_seconds,bot_rating,a_turn_type_Challenge__mean,a_turn_type_Pass__sum_values,a_turn_type_Timeout__mean,a_Z__mean,p_points__sum_values,p_points__median,...,b_l__mean,b_move_len__sum_values,b_jokers_num__mean,time_control_name_blitz,time_control_name_rapid,time_control_name_regular,lexicon_CSW21,rating_mode_CASUAL,bot_name_BetterBot,bot_name_HastyBot
1,1200,1,674.844274,1637,0.0,0.0,0.00000,0.037037,429.0,25.0,...,0.0,60.0,0.000000,0,0,1,0,1,1,0
3,900,5,492.268262,2071,0.0,0.0,0.00000,0.037037,440.0,28.0,...,0.0,57.0,0.076923,0,0,1,1,0,1,0
4,3600,1,350.861141,1936,0.0,0.0,0.00000,0.034483,119.0,5.5,...,0.0,76.0,0.066667,0,0,1,1,1,1,0
5,1200,1,642.688722,1844,0.0,0.0,0.00000,0.032258,325.0,20.5,...,0.0,67.0,0.133333,0,0,1,0,1,0,0
6,900,1,426.950541,2143,0.0,0.0,0.00000,0.040000,378.0,28.5,...,0.0,66.0,0.153846,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72767,3600,1,3669.988413,1952,0.0,0.0,0.03125,0.031250,340.0,16.5,...,0.0,73.0,0.062500,0,0,1,1,1,1,0
72770,3600,1,3662.659287,2237,0.0,0.0,0.03125,0.031250,97.0,6.0,...,0.0,79.0,0.062500,0,0,1,1,0,0,1
72771,900,10,236.133634,1614,0.0,0.0,0.00000,0.032258,390.0,20.5,...,0.0,59.0,0.133333,0,0,1,0,1,0,1
72772,900,1,681.370077,1674,0.0,0.0,0.00000,0.033333,388.0,19.0,...,0.0,64.0,0.000000,0,0,1,0,0,1,0


In [19]:
grid = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0),
    'param_grid':
        {
         'max_depth':range(3,10,2),
         'min_child_weight':range(1,6,2)
        },
    'scores': ['neg_root_mean_squared_error']
    },
}

In [20]:
my_searcher = Searcher(reg_pipe, grid, 2, redo_scores=False)
my_searcher.search(n_data, n_ratings)

In [21]:
res = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher.get_results())

In [22]:
res.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,max_depth,min_child_weight
model_name,,,,,,,,,,,,,,
xgboost,3.064295,0.146605,0.127461,0.017559,-115.421505,-116.412136,-117.457351,-115.402275,-113.731924,-115.685038,1.235623,12,3,3
xgboost,3.086536,0.145000,0.105763,0.008291,-114.566101,-116.408222,-117.293965,-116.207259,-113.573980,-115.609905,1.346456,11,3,1
xgboost,2.839121,0.106086,0.101230,0.002719,-115.003279,-116.663357,-116.705893,-115.506285,-113.408597,-115.457482,1.218136,10,3,5
xgboost,4.316084,0.024338,0.098368,0.002872,-108.073076,-109.801571,-109.916573,-108.604507,-108.511328,-108.981411,0.739635,9,5,1
xgboost,4.829890,0.143510,0.114811,0.011810,-108.448240,-109.297261,-109.803870,-108.964112,-108.122886,-108.927274,0.597191,8,5,3
xgboost,5.131907,0.088755,0.116143,0.002403,-107.851393,-109.307946,-110.519722,-108.194734,-107.506128,-108.675985,1.102620,7,5,5
xgboost,12.192096,0.439086,0.141303,0.009168,-107.095032,-109.279269,-109.241556,-109.153733,-107.023934,-108.358705,1.061830,6,9,1
xgboost,11.480485,1.342206,0.127457,0.020886,-106.761544,-108.924066,-109.517539,-107.477409,-106.671236,-107.870359,1.152847,5,9,5
xgboost,8.854729,0.130835,0.132711,0.005915,-106.260886,-107.206620,-108.815623,-107.904239,-106.956857,-107.428845,0.870108,4,7,3


In [23]:
grid2 = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0),
    'param_grid':
        {
         'max_depth':[6, 7, 8],
         'min_child_weight':[1, 2, 3]
        },
    'scores': ['neg_root_mean_squared_error']
    },
}
my_searcher2 = Searcher(reg_pipe, grid2, 2, redo_scores=False)
my_searcher2.search(n_data, n_ratings)
res2 = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher2.get_results())
res2.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,max_depth,min_child_weight
model_name,,,,,,,,,,,,,,
xgboost,5.583624,0.202048,0.106798,0.006976,-106.555618,-107.884547,-109.541622,-107.104168,-107.027534,-107.622698,1.050008,9,6,3
xgboost,5.326306,0.136686,0.103382,0.006549,-106.659047,-107.797058,-110.294750,-106.656247,-106.475194,-107.576459,1.437984,8,6,2
xgboost,9.956766,1.617820,0.125590,0.016430,-106.555366,-107.591758,-108.682376,-107.732509,-106.640263,-107.440454,0.784123,7,8,3
xgboost,8.318860,0.293852,0.117970,0.007983,-106.260886,-107.206620,-108.815623,-107.904239,-106.956857,-107.428845,0.870108,6,7,3
xgboost,5.275766,0.031522,0.104014,0.010894,-106.930830,-107.452760,-109.006665,-107.826429,-105.840514,-107.411440,1.040909,5,6,1
xgboost,10.084830,0.103930,0.133036,0.003215,-106.380639,-108.174448,-108.925564,-107.270702,-105.783967,-107.307064,1.144666,4,8,2
xgboost,10.760665,0.624821,0.135269,0.003505,-105.594843,-107.360436,-109.037024,-108.381504,-106.072436,-107.289249,1.311827,3,8,1
xgboost,8.034874,0.136949,0.127143,0.008779,-106.790098,-107.634720,-107.083514,-107.679245,-105.772211,-106.991957,0.696049,2,7,2
xgboost,7.422698,0.259456,0.117156,0.008959,-104.783926,-107.408662,-107.371477,-107.227624,-105.996399,-106.557618,1.029223,1,7,1


In [25]:
grid3 = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0, max_depth=7, min_child_weight=1),
    'param_grid':
        {
            'gamma':[i/10.0 for i in range(0,5)]
        },
    'scores': ['neg_root_mean_squared_error']
    },
}
my_searcher3 = Searcher(reg_pipe, grid3, 2, redo_scores=False)
my_searcher3.search(n_data, n_ratings)
res3 = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher3.get_results())
res3.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,gamma
model_name,,,,,,,,,,,,,
xgboost,6.504428,0.087208,0.117396,0.006387,-105.203564,-107.408662,-107.371477,-107.227618,-105.996393,-106.641543,0.888680,5,0.1
xgboost,8.101948,1.392984,0.126215,0.013140,-105.203564,-107.190677,-107.371477,-107.227618,-105.996400,-106.597947,0.854673,4,0.4
xgboost,7.580606,0.287932,0.133590,0.009856,-105.203564,-107.190677,-107.371477,-107.227618,-105.996393,-106.597946,0.854674,2,0.2
xgboost,8.676496,0.359765,0.134216,0.008554,-105.203564,-107.190677,-107.371477,-107.227618,-105.996393,-106.597946,0.854674,2,0.3
xgboost,6.398949,0.024232,0.115319,0.010142,-104.783926,-107.408662,-107.371477,-107.227624,-105.996399,-106.557618,1.029223,1,0.0


In [27]:
grid4 = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0, max_depth=7, min_child_weight=1, gamma=0),
    'param_grid':
        {
            'subsample':[i/10.0 for i in range(6,11)],
             'colsample_bytree':[i/10.0 for i in range(6,11)]
        },
    'scores': ['neg_root_mean_squared_error']
    },
}
my_searcher4 = Searcher(reg_pipe, grid4, 2, redo_scores=False)
my_searcher4.search(n_data, n_ratings)
res4 = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher4.get_results())
res4.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,colsample_bytree,subsample
model_name,,,,,,,,,,,,,,
xgboost,7.958876,0.081760,0.140687,0.027874,-110.850986,-110.752382,-113.584362,-111.513090,-110.609265,-111.462017,1.105539,25,0.8,0.6
xgboost,4.710350,0.279688,0.092290,0.007313,-110.457951,-111.353536,-113.177193,-111.622103,-109.962541,-111.314665,1.106999,24,0.6,0.6
xgboost,8.615715,0.061932,0.126227,0.003279,-110.019791,-112.413155,-112.136451,-111.229067,-109.277638,-111.015221,1.205585,23,0.9,0.6
xgboost,6.751260,0.137414,0.122086,0.005770,-111.400091,-110.462917,-112.614611,-111.110084,-109.436706,-111.004882,1.049894,22,0.7,0.6
xgboost,9.337016,0.066967,0.129443,0.005669,-110.374422,-111.904142,-111.630133,-110.422181,-109.255416,-110.717259,0.957458,21,1.0,0.6
xgboost,7.831178,0.039272,0.123679,0.003617,-109.780162,-109.773739,-111.322967,-110.775818,-109.521104,-110.234758,0.693605,20,0.8,0.7
xgboost,4.408145,0.210912,0.091654,0.011468,-109.122181,-110.241060,-111.594789,-110.761513,-107.950478,-109.934004,1.275201,19,0.6,0.7
xgboost,7.183582,0.247921,0.126680,0.003915,-109.037311,-110.185027,-111.779805,-109.345008,-109.278697,-109.925170,1.004988,18,0.7,0.7
xgboost,8.698079,0.283705,0.129970,0.006204,-108.830968,-110.716993,-110.850918,-109.541902,-108.886463,-109.765449,0.869428,17,0.9,0.7


In [28]:
grid5 = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0, max_depth=7, min_child_weight=1, gamma=0),
    'param_grid':
        {
            'subsample':[i/100.0 for i in range(80,101, 5)],
             'colsample_bytree':[i/100.0 for i in range(80,101, 5)]
        },
    'scores': ['neg_root_mean_squared_error']
    },
}
my_searcher5 = Searcher(reg_pipe, grid5, 2, redo_scores=False)
my_searcher5.search(n_data, n_ratings)
res5 = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher5.get_results())
res5.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,colsample_bytree,subsample
model_name,,,,,,,,,,,,,,
xgboost,8.511759,0.102506,0.131866,0.008343,-108.324073,-109.795941,-110.401211,-109.345295,-107.847448,-109.142793,0.938024,25,0.9,0.8
xgboost,8.294712,0.368211,0.116577,0.004815,-108.272922,-109.950357,-109.225635,-109.974004,-107.901185,-109.064820,0.850624,24,0.9,0.85
xgboost,8.346015,0.372889,0.126800,0.005948,-107.954132,-109.566012,-110.737159,-109.355101,-107.526618,-109.027805,1.159743,23,0.85,0.8
xgboost,8.704278,0.084348,0.114584,0.004045,-108.333863,-109.014764,-110.611838,-108.937287,-107.282612,-108.836073,1.082679,22,1.0,0.8
xgboost,8.750831,0.160506,0.125902,0.002677,-106.504288,-109.296271,-109.859642,-109.443237,-108.585984,-108.737884,1.189931,21,0.95,0.85
xgboost,5.874143,0.210925,0.105732,0.009992,-107.000258,-109.950852,-109.984182,-109.390610,-106.967134,-108.658607,1.383769,20,0.8,0.8
xgboost,8.871281,0.146806,0.128100,0.003825,-107.515303,-109.806949,-110.591895,-108.253254,-106.210341,-108.475548,1.571891,19,0.95,0.8
xgboost,5.639898,0.040116,0.107033,0.007773,-106.625986,-108.361121,-109.554952,-109.799164,-107.237686,-108.315782,1.245449,18,0.8,0.85
xgboost,8.028964,0.052780,0.127919,0.004325,-107.555719,-109.006094,-109.303098,-107.886191,-107.786120,-108.307444,0.706224,17,0.85,0.85


In [34]:
grid6 = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0, max_depth=7, min_child_weight=1, gamma=0, subsample=1, colsample_bytree=1),
    'param_grid':
        {
            'reg_alpha':[0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 10, 100],
            'reg_lambda':[0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 10, 100]
        },
    'scores': ['neg_root_mean_squared_error']
    },
}
my_searcher6 = Searcher(reg_pipe, grid6, 2, redo_scores=False)
my_searcher6.search(n_data, n_ratings)
res6 = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher6.get_results())
res6.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,reg_alpha,reg_lambda
model_name,,,,,,,,,,,,,,
xgboost,9.878621,0.585853,0.144858,0.028972,-108.194597,-109.045693,-108.855036,-107.661683,-106.957268,-108.142855,0.769694,81,0.1,100
xgboost,8.069425,0.045317,0.116780,0.004225,-108.194569,-109.045673,-108.854998,-107.661642,-106.957238,-108.142824,0.769696,80,0.0001,100
xgboost,8.677541,0.155640,0.124984,0.004377,-108.194567,-109.045673,-108.854997,-107.661641,-106.957238,-108.142823,0.769695,79,0.00001,100
xgboost,8.954652,0.328369,0.134098,0.009610,-108.194567,-109.045673,-108.854997,-107.661642,-106.957238,-108.142823,0.769695,78,0,100
xgboost,8.583774,0.063384,0.129397,0.007439,-108.194573,-109.045658,-108.854999,-107.661647,-106.957239,-108.142823,0.769691,77,0.01,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xgboost,8.990616,0.210404,0.127157,0.008994,-106.311980,-106.907353,-108.247807,-106.191610,-105.640426,-106.659835,0.890208,5,100,0.1
xgboost,8.655558,0.095957,0.131639,0.004978,-104.783926,-107.408662,-107.371477,-107.227624,-105.996399,-106.557618,1.029223,4,0,1
xgboost,8.799349,0.085433,0.132768,0.005489,-104.783926,-107.408660,-107.371477,-107.227624,-105.996400,-106.557617,1.029222,3,0.00001,1


In [35]:
grid7 = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0, max_depth=7, min_child_weight=1, gamma=0, subsample=1, colsample_bytree=1),
    'param_grid':
        {
            'reg_alpha':[0, 0.0001, 0.0005, 0.001, 0.00005, 0.00001],
            'reg_lambda':[0, 1, 1.5, 10, 0.5, 0.1]

        },
    'scores': ['neg_root_mean_squared_error']
    },
}
my_searcher7 = Searcher(reg_pipe, grid7, 2, redo_scores=False)
my_searcher7.search(n_data, n_ratings)
res7 = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher7.get_results())
res7.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,reg_alpha,reg_lambda
model_name,,,,,,,,,,,,,,
xgboost,9.090482,0.386325,0.127189,0.002783,-107.039913,-107.476254,-109.776960,-107.736714,-106.857537,-107.777476,1.046959,36,0.00001,0
xgboost,9.438260,0.905085,0.128557,0.005967,-107.039912,-107.476253,-109.776959,-107.736713,-106.857537,-107.777475,1.046959,35,0.00005,0
xgboost,6.916713,0.323409,0.105669,0.009501,-107.039912,-107.476254,-109.776954,-107.736715,-106.857538,-107.777475,1.046957,34,0,0
xgboost,9.934885,0.692995,0.133070,0.008434,-107.039907,-107.476262,-109.776955,-107.736712,-106.857537,-107.777475,1.046958,33,0.0001,0
xgboost,9.063685,0.371244,0.127608,0.005552,-107.039903,-107.476257,-109.776955,-107.736705,-106.857507,-107.777465,1.046964,32,0.0005,0
xgboost,10.079825,1.108317,0.136416,0.004142,-107.039900,-107.216255,-109.427075,-107.736693,-106.857497,-107.655484,0.933143,31,0.001,0
xgboost,9.388934,0.286000,0.137551,0.020455,-106.489286,-107.071839,-108.403818,-107.558783,-105.288417,-106.962428,1.046078,30,0.00001,0.5
xgboost,11.342956,2.605247,0.117138,0.010247,-106.489286,-107.071839,-108.403818,-107.558783,-105.288417,-106.962428,1.046078,29,0.00005,0.5
xgboost,8.165734,0.528573,0.122030,0.006583,-106.489286,-107.071839,-108.403817,-107.558783,-105.288417,-106.962428,1.046077,28,0,0.5


In [36]:
grid8 = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0,
                              max_depth=7, min_child_weight=1,
                              gamma=0,
                              subsample=1, colsample_bytree=1,
                              reg_alpha=0.00005, reg_lambda=1
                              ),
    'param_grid':
        {
            'learning_rate':[0.9 / (3 ** i) for i in range(5)]
        },
    'scores': ['neg_root_mean_squared_error']
    },
}
my_searcher8 = Searcher(reg_pipe, grid8, 2, redo_scores=False)
my_searcher8.search(n_data, n_ratings)
res8 = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher8.get_results())
res8.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,learning_rate
model_name,,,,,,,,,,,,,
xgboost,8.508148,1.452793,0.111242,0.019132,-600.067848,-602.972194,-602.829284,-602.275490,-603.638589,-602.356681,1.223997,5,0.011111
xgboost,7.048149,0.350940,0.099713,0.004010,-128.844937,-131.138509,-130.150684,-129.506650,-128.932257,-129.714607,0.851974,4,0.9
xgboost,11.049089,1.995679,0.112001,0.013992,-125.109852,-126.959765,-128.099594,-126.951114,-125.917912,-126.607647,1.018527,3,0.033333
xgboost,6.395107,0.177400,0.107639,0.013126,-104.783926,-107.408660,-107.279476,-106.846093,-105.996400,-106.462911,0.974133,2,0.3
xgboost,9.034054,2.816331,0.109684,0.002546,-105.461633,-106.107754,-107.503204,-106.522715,-104.908161,-106.100693,0.891708,1,0.1


In [37]:
grid9 = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0,
                              max_depth=7, min_child_weight=1,
                              gamma=0,
                              subsample=1, colsample_bytree=1,
                              reg_alpha=0.00005, reg_lambda=1
                              ),
    'param_grid':
        {
            'learning_rate':[0.1 * (np.sqrt(3) ** i) for i in range(-3, 3)]
        },
    'scores': ['neg_root_mean_squared_error']
    },
}
my_searcher9 = Searcher(reg_pipe, grid9, 2, redo_scores=False)
my_searcher9.search(n_data, n_ratings)
res9 = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher9.get_results())
res9.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,learning_rate
model_name,,,,,,,,,,,,,
xgboost,7.289185,1.294005,0.101371,0.003498,-281.606799,-283.984683,-284.445693,-283.861467,-284.020877,-283.583904,1.007992,6,0.019245
xgboost,6.280607,0.116680,0.157346,0.082932,-125.109852,-126.959765,-128.099594,-126.951114,-125.917912,-126.607647,1.018527,5,0.033333
xgboost,8.201463,0.502041,0.129350,0.021756,-107.051805,-107.922696,-109.481399,-107.984232,-107.025424,-107.893111,0.893590,4,0.057735
xgboost,8.808343,0.246133,0.136329,0.008073,-104.783926,-107.408660,-107.279476,-106.846093,-105.996400,-106.462911,0.974133,3,0.3
xgboost,9.061542,0.620388,0.136671,0.010226,-105.461633,-106.107754,-107.503204,-106.522715,-104.908161,-106.100693,0.891708,2,0.1
xgboost,9.180079,0.114310,0.144401,0.014044,-104.618741,-104.913558,-106.587129,-105.158422,-103.947271,-105.045024,0.871207,1,0.173205


In [40]:
np.sqrt(3) / 10

0.17320508075688773

In [42]:
grid10 = {
    'xgboost':
    {
    'estimator': XGBRegressor(random_state=0,
                              max_depth=7, min_child_weight=1,
                              gamma=0,
                              subsample=1, colsample_bytree=1,
                              reg_alpha=0.00005, reg_lambda=1,
                              learning_rate=np.sqrt(3)/10
                              ),
    'param_grid':
        {
            'random_state':[0]
        },
    'scores': ['neg_root_mean_squared_error']
    },
}
my_searcher10 = Searcher(reg_pipe, grid10, 2, redo_scores=False)
my_searcher10.search(n_data, n_ratings)
res10 = SearcherResultsOrgenizer(redo_scores=False).orgenize(my_searcher10.get_results())
res10.sort_values('mean_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_neg_root_mean_squared_error,split1_test_neg_root_mean_squared_error,split2_test_neg_root_mean_squared_error,split3_test_neg_root_mean_squared_error,split4_test_neg_root_mean_squared_error,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,random_state
model_name,,,,,,,,,,,,,
xgboost,6.139761,1.115928,0.110939,0.020002,-104.618741,-104.913558,-106.587129,-105.158422,-103.947271,-105.045024,0.871207,1,0
